# Clustering Storage Modes

Compare different storage handling modes when clustering time series.

This notebook demonstrates:

- **Four storage modes**: `independent`, `cyclic`, `intercluster`, `intercluster_cyclic`
- **Seasonal storage**: Why inter-cluster linking matters for long-term storage
- **When to use each mode**: Choosing the right mode for your application

!!! note "Prerequisites"
    Read [08c-clustering](08c-clustering.ipynb) first for clustering basics.

In [ ]:
import timeit
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import flixopt as fx

fx.CONFIG.notebook()

## Load the Seasonal Storage System

We use a solar thermal + seasonal pit storage system with a full year of data.
This is ideal for demonstrating storage modes because:

- **Solar peaks in summer** when heat demand is low
- **Heat demand peaks in winter** when solar is minimal
- **Seasonal storage** bridges this gap by storing summer heat for winter

In [ ]:
# Generate example data if not present
data_file = Path('data/seasonal_storage_system.nc4')
if not data_file.exists():
    from data.generate_example_systems import create_seasonal_storage_system

    fs = create_seasonal_storage_system()
    fs.to_netcdf(data_file)

# Load the seasonal storage system
flow_system = fx.FlowSystem.from_netcdf(data_file)

timesteps = flow_system.timesteps
print(f'Loaded FlowSystem: {len(timesteps)} timesteps ({len(timesteps) / 24:.0f} days)')
print(f'Components: {list(flow_system.components.keys())}')

In [ ]:
# Visualize the seasonal patterns
solar_profile = flow_system.components['SolarThermal'].outputs[0].fixed_relative_profile
heat_demand = flow_system.components['HeatDemand'].inputs[0].fixed_relative_profile

# Daily average for clearer visualization
solar_daily = solar_profile.values.reshape(-1, 24).mean(axis=1)
demand_daily = heat_demand.values.reshape(-1, 24).mean(axis=1)
days = np.arange(len(solar_daily))

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1)
fig.add_trace(go.Scatter(x=days, y=solar_daily, name='Solar (daily avg)', fill='tozeroy'), row=1, col=1)
fig.add_trace(go.Scatter(x=days, y=demand_daily, name='Heat Demand (daily avg)', fill='tozeroy'), row=2, col=1)
fig.update_layout(height=400, title='Seasonal Mismatch: Solar vs Heat Demand')
fig.update_xaxes(title_text='Day of Year', row=2, col=1)
fig.update_yaxes(title_text='Solar Profile', row=1, col=1)
fig.update_yaxes(title_text='Heat Demand [MW]', row=2, col=1)
fig.show()

## Understanding Storage Modes

When clustering reduces a full year to typical periods (e.g., 12 typical days), we need to
decide how storage behaves across these periods. Each `Storage` component has a 
`cluster_storage_mode` parameter with four options:

| Mode | Description | Use Case |
|------|-------------|----------|
| `'intercluster_cyclic'` | Links storage across clusters + yearly cyclic | **Default**. Seasonal storage, yearly optimization |
| `'intercluster'` | Links storage across clusters, free start/end | Multi-year optimization, flexible boundaries |
| `'cyclic'` | Each cluster independent, but cyclic (start = end) | Daily storage only, no seasonal effects |
| `'independent'` | Each cluster independent, free start/end | Fastest solve, ignores long-term storage |

Let's compare them!

## Baseline: Full Year Optimization

First, optimize the full system to establish a baseline:

In [ ]:
solver = fx.solvers.HighsSolver(mip_gap=0.02)

start = timeit.default_timer()
fs_full = flow_system.copy()
fs_full.optimize(solver)
time_full = timeit.default_timer() - start

print(f'Full optimization: {time_full:.1f} seconds')
print(f'Total cost: {fs_full.solution["costs"].item():,.0f} EUR')
print('\nOptimized sizes:')
for name, size in fs_full.statistics.sizes.items():
    print(f'  {name}: {float(size.item()):.2f}')

## Compare Storage Modes

Now let's cluster with each storage mode and compare results.
We set `cluster_storage_mode` on the Storage component before calling `cluster()`:

In [ ]:
# Clustering parameters
N_CLUSTERS = 12  # 12 typical days for a full year
CLUSTER_DURATION = '1D'
PEAK_SERIES = ['HeatDemand(Q_th)|fixed_relative_profile']

# Storage modes to compare
storage_modes = ['independent', 'cyclic', 'intercluster', 'intercluster_cyclic']

results = {}
clustered_systems = {}

for mode in storage_modes:
    print(f'\n--- Mode: {mode} ---')

    # Create a copy and set the storage mode
    fs_copy = flow_system.copy()
    fs_copy.storages['SeasonalStorage'].cluster_mode = mode

    start = timeit.default_timer()
    fs_clustered = fs_copy.transform.cluster(
        n_clusters=N_CLUSTERS,
        cluster_duration=CLUSTER_DURATION,
        time_series_for_high_peaks=PEAK_SERIES,
    )
    time_cluster = timeit.default_timer() - start

    start = timeit.default_timer()
    fs_clustered.optimize(solver)
    time_solve = timeit.default_timer() - start

    clustered_systems[mode] = fs_clustered

    results[mode] = {
        'Time [s]': time_cluster + time_solve,
        'Cost [EUR]': fs_clustered.solution['costs'].item(),
        'Solar [MW]': fs_clustered.statistics.sizes.get('SolarThermal(Q_th)', 0),
        'Boiler [MW]': fs_clustered.statistics.sizes.get('GasBoiler(Q_th)', 0),
        'Storage [MWh]': fs_clustered.statistics.sizes.get('SeasonalStorage', 0),
    }

    # Handle xarray types
    for key in ['Solar [MW]', 'Boiler [MW]', 'Storage [MWh]']:
        val = results[mode][key]
        results[mode][key] = float(val.item()) if hasattr(val, 'item') else float(val)

    print(f'  Time: {results[mode]["Time [s]"]:.1f}s')
    print(f'  Cost: {results[mode]["Cost [EUR]"]:,.0f} EUR')
    print(f'  Storage: {results[mode]["Storage [MWh]"]:.0f} MWh')

In [ ]:
# Add full optimization result for comparison
results['Full (baseline)'] = {
    'Time [s]': time_full,
    'Cost [EUR]': fs_full.solution['costs'].item(),
    'Solar [MW]': float(fs_full.statistics.sizes.get('SolarThermal(Q_th)', 0).item()),
    'Boiler [MW]': float(fs_full.statistics.sizes.get('GasBoiler(Q_th)', 0).item()),
    'Storage [MWh]': float(fs_full.statistics.sizes.get('SeasonalStorage', 0).item()),
}

# Create comparison DataFrame
comparison = pd.DataFrame(results).T
baseline_cost = comparison.loc['Full (baseline)', 'Cost [EUR]']
baseline_time = comparison.loc['Full (baseline)', 'Time [s]']
comparison['Cost Gap [%]'] = (comparison['Cost [EUR]'] - baseline_cost) / abs(baseline_cost) * 100
comparison['Speedup'] = baseline_time / comparison['Time [s]']

comparison.style.format(
    {
        'Time [s]': '{:.1f}',
        'Cost [EUR]': '{:,.0f}',
        'Solar [MW]': '{:.1f}',
        'Boiler [MW]': '{:.1f}',
        'Storage [MWh]': '{:.0f}',
        'Cost Gap [%]': '{:+.1f}',
        'Speedup': '{:.1f}x',
    }
)

## Visualize Storage Behavior

The key difference between modes is how storage is utilized across the year.
Let's expand each solution back to full resolution and compare:

In [ ]:
# Expand clustered solutions to full resolution
expanded_systems = {}
for mode in storage_modes:
    expanded_systems[mode] = clustered_systems[mode].transform.expand_solution()

In [ ]:
# Plot storage charge state for each mode
fig = make_subplots(
    rows=len(storage_modes) + 1,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=['Full Optimization'] + [f'Mode: {m}' for m in storage_modes],
)

# Full optimization
soc_full = fs_full.solution['SeasonalStorage|charge_state']
fig.add_trace(go.Scatter(x=fs_full.timesteps, y=soc_full.values, name='Full', line=dict(width=0.8)), row=1, col=1)

# Expanded clustered solutions
for i, mode in enumerate(storage_modes, start=2):
    fs_exp = expanded_systems[mode]
    soc = fs_exp.solution['SeasonalStorage|charge_state']
    fig.add_trace(go.Scatter(x=fs_exp.timesteps, y=soc.values, name=mode, line=dict(width=0.8)), row=i, col=1)

fig.update_layout(height=800, title='Storage Charge State by Mode', showlegend=False)
for i in range(1, len(storage_modes) + 2):
    fig.update_yaxes(title_text='SOC [MWh]', row=i, col=1)
fig.show()

## Interpretation

### `'independent'` Mode
- Each typical period is solved independently
- Storage starts and ends at arbitrary states within each cluster
- **No seasonal storage benefit captured** - storage is only used for daily fluctuations
- Fastest to solve but least accurate for seasonal systems

### `'cyclic'` Mode  
- Each cluster is independent but enforces start = end state
- Better than independent but still **no cross-season linking**
- Good for systems where storage only balances within-day variations

### `'intercluster'` Mode
- Links storage state across the original time series via typical periods
- **Captures seasonal storage behavior** - summer charging, winter discharging
- Free start and end states (useful for multi-year optimization)

### `'intercluster_cyclic'` Mode (Default)
- Inter-cluster linking **plus** yearly cyclic constraint (end = start)
- **Best for yearly investment optimization** with seasonal storage
- Ensures the storage cycle is sustainable year after year

## When to Use Each Mode

| Your System Has... | Recommended Mode |
|-------------------|------------------|
| Seasonal storage (pit, underground) | `'intercluster_cyclic'` |
| Only daily storage (batteries, hot water tanks) | `'cyclic'` |
| Multi-year optimization with inter-annual storage | `'intercluster'` |
| Quick sizing estimate, storage not critical | `'independent'` |

### Setting the Mode

```python
# Option 1: Set when creating the Storage
storage = fx.Storage(
    'SeasonalStorage',
    capacity_in_flow_hours=5000,
    cluster_storage_mode='intercluster_cyclic',  # default
    ...
)

# Option 2: Modify before clustering
flow_system.components['SeasonalStorage'].cluster_storage_mode = 'cyclic'
fs_clustered = flow_system.transform.cluster(...)
```

!!! tip "Rule of Thumb"
    Use `'intercluster_cyclic'` (default) unless you have a specific reason not to.
    It provides the most accurate representation of storage behavior in clustered systems.

## Summary

You learned how to:

- Use **`cluster_storage_mode`** on Storage components to control behavior in clustering
- Understand the **difference between modes** and their impact on results
- Choose the **right mode** for your optimization problem

### Key Takeaways

1. **Seasonal storage requires inter-cluster linking** to capture charging/discharging across seasons
2. **`'intercluster_cyclic'`** is the default and best for yearly investment optimization
3. **`'independent'` and `'cyclic'`** are faster but miss long-term storage value
4. **Expand solutions** with `expand_solution()` to visualize storage behavior across the year

### Next Steps

- **[08d-clustering-multiperiod](08d-clustering-multiperiod.ipynb)**: Clustering with multiple periods and scenarios